In [41]:
# Load model
import os
script_path = os.getcwd()
from cobra.io import read_sbml_model, write_sbml_model
#path = 'Chlamy_model_iCre1355-master/iCre1355_SBML_Matlab_files/iCre1355_auto.xml'
path_sbml3fbc = 'iCre1355_auto_fbc.xml'
#model = read_sbml_model(path)
#write_sbml_model(model, path_sbml3fbc)  # stores as SBML3 + fbc
model = read_sbml_model(os.path.join(script_path, path_sbml3fbc))  # magic, no warnings

# NOTE: I had to re-write the model because it gave me A LOT of warnings and made everything go super slow.

No objective coefficients in model. Unclear what should be optimized


In [42]:
model

Name,iCre1355
Memory address,0x07fa690653c10
Number of metabolites,1845
Number of reactions,2394
Number of groups,117
Objective expression,0
Compartments,"Cytosol, Chloroplast, Mitochondria, Glyoxysome, Flagellum, Extra-organism, Nucleus, Golgi Apparatus, Eyespot, Thylakoid Lumen, Inner Mitochondrial membrane space"


In [9]:
# Assess model with memote

!memote report snapshot --filename "iCre1355_auto_fbc.html" "/Users/ericbf10/Library/CloudStorage/GoogleDrive-ericbaufa10@gmail.com/My Drive/PERSONAL/DTU/DTU - Bioinformatics and Systems Biology/2022_Autumn_Semester/27410_Computer-Aided_Cell_Factory_Design/Project/iCre1355_auto_fbc.xml"

error: No objective coefficients in model. Unclear what should be optimized
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
============================= test session starts ==============================
platform darwin -- Python 3.8.2, pytest-4.6.11, py-1.11.0, pluggy-0.13.1
rootdir: /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/memote/suite/tests
plugins: anyio-3.6.1
collected 164 items / 1 skipped / 163 selected                                 

../../../../../../../../../../../../Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/memote/suite/tests/test_annotation.py F [  0%]
FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF..         [ 39%]
../../../../../../../../../../../../Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/memote/suite/tests/test_basic.py . [ 40%]
.....F........F.....FF                             

In [10]:
!memote report snapshot --filename "iRC1080.html" "/Users/ericbf10/Library/CloudStorage/GoogleDrive-ericbaufa10@gmail.com/My Drive/PERSONAL/DTU/DTU - Bioinformatics and Systems Biology/2022_Autumn_Semester/27410_Computer-Aided_Cell_Factory_Design/Project/iRC1080.xml"

error: No objective coefficients in model. Unclear what should be optimized
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
============================= test session starts ==============================
platform darwin -- Python 3.8.2, pytest-4.6.11, py-1.11.0, pluggy-0.13.1
rootdir: /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/memote/suite/tests
plugins: anyio-3.6.1
collected 164 items / 1 skipped / 163 selected                                 

../../../../../../../../../../../../Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/memote/suite/tests/test_annotation.py . [  0%]
..FFFFFFFF..FFFFF.FFFFFFFFFFFFFFF..FF..F...F....FFF.FFFFFFFFFF..         [ 39%]
../../../../../../../../../../../../Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/memote/suite/tests/test_basic.py . [ 40%]
.....F........F...F.FF                             

In [53]:
# Explore the model

print("The model has %d reactions and %d metabolites" % (len(model.reactions), len(model.metabolites)))

The model has 2394 reactions and 1845 metabolites


In [54]:
# Create the stoichiometric matrix

from cobra.util import create_stoichiometric_matrix
S = create_stoichiometric_matrix(model)
print(S)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [55]:
# Check non-zero elements of S

import numpy as np
non_zero_elem = np.count_nonzero(S)
zero_elem = np.count_nonzero(S==0)
print("The percentage of non-zero elements in S is:", str(non_zero_elem/(zero_elem + non_zero_elem)*100) + "%")

The percentage of non-zero elements in S is: 0.2494492781185122%


In [56]:
for reaction in model.reactions.query('Biomass', 'name'):
    print(reaction.id)

Biomass_Chlamy_auto
Biomass_Chlamy_mixo
Biomass_Chlamy_hetero


In [57]:
# Add objective function

model.objective = 'Biomass_Chlamy_auto'

In [58]:
# Simulate the model

solution = model.optimize()
solution

,fluxes,reduced_costs
AAT,0.000011,0.000000e+00
ALATLm,0.000000,0.000000e+00
ASNTLm,0.000000,0.000000e+00
ASPO,0.000000,-1.110223e-16
ASPOm,0.000000,-1.110223e-16
...,...,...
PYDXPP,0.000000,0.000000e+00
PYDXS,0.000000,0.000000e+00
Biomass_Chlamy_auto,0.051575,-2.657596e-15
Biomass_Chlamy_mixo,0.000000,-2.020611e+00


In [60]:
# Change medium composition (acetate)

medium = model.medium
medium['EX_ac_e'] = 1000
model.medium = medium
model.medium

{'EX_h_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_nh4_e': 1.0,
 'EX_so4_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_na1_e': 10.0,
 'EX_photonVis_e': 80.0,
 'EX_o2_e': 10.0,
 'EX_co2_e': 2.0,
 'EX_ac_e': 1000}

In [61]:
# Add heterologous reactions and metabolites

from cobra import Reaction, Metabolite

### Create metabolite ###
metabolite1 = Metabolite('jet_juice_c')
metabolite1.name = 'Jet Juice'
metabolite1.formula = 'C29H38O2'
metabolite1.compartment = 'c'
model.add_metabolites([metabolite1])

### Define 1st reaction ###
reaction1 = Reaction('jet_fuel')
reaction1.name = 'iPKS'
reaction1.lower_bound = 0.  # This makes the reaction irreversible
reaction1.upper_bound = 1000.  # This is the default

# Add metabolites to the reaction
reaction1.add_metabolites({
    model.metabolites.get_by_id("ac_c"): -11.0,
    model.metabolites.get_by_id("nadp_c"): 11.0,
    model.metabolites.get_by_id("amet_c"): -7.0,
    model.metabolites.get_by_id("ahcys_c"): 7.0,
    model.metabolites.get_by_id("nadph_c"): -11.0,
    model.metabolites.get_by_id("h_c"): 11.0,
    model.metabolites.get_by_id("jet_juice_c"): 1.0
})

### Define 2nd reaction ###
reaction2 = Reaction('EX_jet_fuel')
reaction2.name = 'Jet Fuel exchange'
reaction2.lower_bound = 0.  # This makes the reaction irreversible
reaction2.upper_bound = 1000.  # This is the default

# Add metabolites to the reaction
reaction2.add_metabolites({
    model.metabolites.get_by_id("jet_juice_c"): -1.0
})

model.add_reactions([reaction1, reaction2])

print(reaction1.reaction)
print(reaction2.reaction)

11.0 ac_c + 7.0 amet_c + 11.0 nadph_c --> 7.0 ahcys_c + 11.0 h_c + jet_juice_c + 11.0 nadp_c
jet_juice_c --> 


In [50]:
## DEBUGGING ##

for reaction in model.metabolites.ac_e.reactions:
    print(reaction.id, reaction, reaction.name, "\n")

ACt ACt: ac_e <=> ac_c acetate transport in via proton symport 

EX_ac_e EX_ac_e: ac_e -->  Acetate exchange 



In [52]:
model.reactions.get_by_id('ACt')

Reaction identifier,ACt
Name,acetate transport in via proton symport
Memory address,0x7fa69389e2b0
Stoichiometry,ac_e <=> ac_c Acetate <=> Acetate
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [165]:
model.reactions.get_by_id('EX_ac_e')

Reaction identifier,EX_ac_e
Name,Acetate exchange
Memory address,0x7f86402226a0
Stoichiometry,ac_e --> Acetate -->
GPR,
Lower bound,0.0
Upper bound,0.0


In [62]:
# Add objective function

model.objective = 'jet_fuel'

In [63]:
# Simulate the model

solution = model.optimize()
solution

,fluxes,reduced_costs
AAT,0.000000,0.000000e+00
ALATLm,0.000000,-1.110223e-16
ASNTLm,0.000000,-1.110223e-16
ASPO,0.000000,2.220446e-16
ASPOm,0.000000,1.110223e-16
...,...,...
Biomass_Chlamy_auto,0.000000,-2.398696e+01
Biomass_Chlamy_mixo,0.000000,-2.427155e+01
Biomass_Chlamy_hetero,0.000000,-9.100529e+00
jet_fuel,9.648802,2.997602e-15


In [88]:
# Change medium composition (acetate)

medium = model.medium
medium['EX_photonVis_e'] = 0
model.medium = medium
model.medium

{'EX_h_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_nh4_e': 1.0,
 'EX_so4_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_na1_e': 10.0,
 'EX_o2_e': 10.0,
 'EX_ac_e': 10.0}

In [89]:
# Simulate the model

solution = model.optimize()
solution

,fluxes,reduced_costs
AAT,6.910494,-1.387779e-17
ALATLm,0.000000,0.000000e+00
ASNTLm,0.000000,0.000000e+00
ASPO,0.000000,-1.958864e-02
ASPOm,0.000000,-1.958864e-02
...,...,...
Biomass_Chlamy_auto,0.000000,-2.753142e+00
Biomass_Chlamy_mixo,0.000000,-2.817148e+00
Biomass_Chlamy_hetero,0.000000,-1.037483e+00
jet_fuel,0.439066,0.000000e+00


In [86]:
# Change medium composition (acetate)

medium = model.medium
medium['EX_co2_e'] = 0
model.medium = medium
model.medium

{'EX_h_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_nh4_e': 1.0,
 'EX_so4_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_na1_e': 10.0,
 'EX_photonVis_e': 80.0,
 'EX_o2_e': 10.0,
 'EX_ac_e': 10.0}

In [87]:
# Simulate the model

solution = model.optimize()
solution

,fluxes,reduced_costs
AAT,-1.371718,-1.387779e-17
ALATLm,0.000000,4.163336e-17
ASNTLm,0.000000,2.775558e-17
ASPO,1.988208,1.387779e-17
ASPOm,0.000000,1.387779e-17
...,...,...
Biomass_Chlamy_auto,0.000000,-1.703488e+00
Biomass_Chlamy_mixo,0.000000,-1.652603e+00
Biomass_Chlamy_hetero,0.000000,-7.792261e-01
jet_fuel,0.689655,1.526557e-16


In [83]:
# Change medium composition (acetate)

medium = model.medium
medium['EX_ac_e'] = 10.
model.medium = medium
model.medium

{'EX_h_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_nh4_e': 1.0,
 'EX_so4_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_na1_e': 10.0,
 'EX_photonVis_e': 80.0,
 'EX_o2_e': 10.0,
 'EX_co2_e': 2.0,
 'EX_ac_e': 10.0}

In [84]:
# Simulate the model

solution = model.optimize()
solution

,fluxes,reduced_costs
AAT,-0.317413,-2.775558e-17
ALATLm,0.000000,-3.700743e-17
ASNTLm,0.000000,0.000000e+00
ASPO,0.000000,-9.313538e-16
ASPOm,0.000000,-9.313538e-16
...,...,...
Biomass_Chlamy_auto,0.000000,-1.703488e+00
Biomass_Chlamy_mixo,0.000000,-1.652603e+00
Biomass_Chlamy_hetero,0.000000,-7.792261e-01
jet_fuel,0.758621,0.000000e+00


In [64]:
# Add objective function

model.objective = 'Biomass_Chlamy_auto'

In [66]:
# Change medium composition (acetate)
from cobra.medium import minimal_medium

max_growth = model.slim_optimize()
minimal_medium(model, max_growth)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cobra/medium/minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()


EX_pi_e            0.084852
EX_nh4_e           1.000000
EX_so4_e           0.004462
EX_mg2_e           0.003959
EX_photonVis_e     0.017931
EX_o2_e            6.882233
EX_ac_e           13.159883
dtype: float64

In [67]:
# Add objective function

model.objective = 'jet_fuel'

In [85]:
# Change medium composition (acetate)
from cobra.medium import minimal_medium

max_fuel = model.slim_optimize()
print(max_fuel)
minimal_medium(model, max_fuel, minimize_components=True)

0.758620689655166


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cobra/medium/minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()


EX_h2o_e           9.989626
EX_na1_e           8.303332
EX_photonVis_e    80.000000
EX_o2_e            4.116881
EX_co2_e           2.000000
EX_ac_e           10.000000
dtype: float64

In [14]:
# Simulate the model

solution = model.optimize()
solution

,fluxes,reduced_costs
AAT,0.000000,-0.000000e+00
ALATLm,0.000000,8.326673e-17
ASNTLm,0.000000,1.110223e-16
ASPO,0.000000,-5.993031e-01
ASPOm,0.000000,-5.993031e-01
...,...,...
Biomass_Chlamy_auto,0.000000,-2.325333e+01
Biomass_Chlamy_mixo,0.000000,-2.373817e+01
Biomass_Chlamy_hetero,0.000000,-8.479169e+00
jet_fuel,8.268467,2.664535e-15
